# Asignaciones financieras


## Datos de proyecto en análisis


In [1]:
proyecto="callacalla"
mes=5
anio=2025

## Librerias necesarias


In [30]:
import pandas as pd
import os
import pprint
import pickle
import json
import xlsxwriter
from xlsxwriter.utility import xl_range, xl_rowcol_to_cell
import re
from typing import Dict, List, Any
import itertools
from pathlib import Path
from dotenv import load_dotenv, find_dotenv
from google.oauth2 import service_account
from google.cloud import firestore
import excel2img
from dotenv import load_dotenv, find_dotenv
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter

## Funciones utiles

### Calculadora de costo total

In [3]:
from typing import TypedDict

class CostoIngenieriaResult(TypedDict):
    costo_directo: float
    gastos_generales: float
    utilidad: float
    subtotal: float
    igv: float
    total: float

def calculadora_costo_total(costo_directo: float) -> CostoIngenieriaResult:
    """
    Calcula el costo total de un proyecto de ingeniería civil en soles peruanos.
    
    Args:
        costo_directo (float): Costo directo del proyecto
        
    Returns:
        CostoIngenieriaResult: Diccionario con todos los valores calculados
    """
    # Calcular gastos generales (10% del costo directo)
    gastos_generales = round(costo_directo * 0.10, 2)
    
    # Calcular utilidad (5% del costo directo)
    utilidad = round(costo_directo * 0.05, 2)
    
    # Calcular subtotal
    subtotal = round(costo_directo + gastos_generales + utilidad, 2)
    
    # Calcular IGV (18% del subtotal)
    igv = round(subtotal * 0.18, 2)
    
    # Calcular total
    total = round(subtotal + igv, 2)
    
    return {
        "costo_directo": round(costo_directo, 2),
        "gastos_generales": gastos_generales,
        "utilidad": utilidad,
        "subtotal": subtotal,
        "igv": igv,
        "total": total
    }


#### Multiplicar el precio unitario actualizado por la carga trabajo

In [4]:
def calcular_costos_unitarios(dict_precios_unitarios_actualizados,
                              cargas_trabajo_contratista):
    """
    Multiplica cada carga de trabajo del contratista por su precio unitario
    correspondiente.

    Args:
        dict_precios_unitarios_actualizados (dict): mapea código → precio unitario.
        cargas_trabajo_contratista (dict): mapea código → carga de trabajo.

    Returns:
        dict: mapea código → precio total (precio unitario * carga).

    Raises:
        KeyError: si alguna clave de cargas_trabajo_contratista no existe en
                  dict_precios_unitarios_actualizados.
    """
    # Comprobar que no falte ninguna clave
    faltantes = set(cargas_trabajo_contratista) - set(
        dict_precios_unitarios_actualizados
    )
    if faltantes:
        raise KeyError(
            f"Faltan precios unitarios para las claves: {faltantes}"
        )

    # Generar el diccionario resultado
    resultado = {
        clave: dict_precios_unitarios_actualizados[clave] * cargas_trabajo_contratista[clave]
        for clave in cargas_trabajo_contratista
    }
    return resultado

## Carga de datos


### Firebase


In [5]:
# 1. Busca el .env en el directorio actual o en cualquiera de los padres
dotenv_path = find_dotenv()
if not dotenv_path:
    raise FileNotFoundError("No se encontró ningún archivo .env en este directorio ni en sus padres.")
load_dotenv(dotenv_path)

# 2. Define el root del proyecto como la carpeta que contiene el .env
project_root = Path(dotenv_path).parent

# 3. Obtén la ruta relativa de las credenciales desde la variable de entorno
rel_cred_path = os.getenv("FIRESTORE_CREDENTIALS")
if not rel_cred_path:
    raise RuntimeError("No existe la variable FIRESTORE_CREDENTIALS en el .env")

# 4. Construye la ruta absoluta al JSON
cred_path = Path(rel_cred_path)
if not cred_path.is_absolute():
    cred_path = (project_root / cred_path).resolve()

if not cred_path.exists():
    raise FileNotFoundError(f"No existe el archivo de credenciales en: {cred_path}")

# 5. Carga las credenciales y crea el cliente de Firestore
credentials = service_account.Credentials.from_service_account_file(str(cred_path))
client = firestore.Client(credentials=credentials, project=credentials.project_id)

# 6. Prueba que funcione
print("Colecciones disponibles:", [c.id for c in client.collections()])

Colecciones disponibles: ['rutinarios']


In [6]:
db=firestore.Client(credentials=credentials, project=credentials.project_id)

In [7]:
# 4. Define la ruta a tu documento anidado
colec_raiz = "rutinarios"
doc_proyecto = proyecto     # puede ser tu variable proyecto
colec_valoriz = "presupuestos"
id_valoriz = "desembolsos"  

doc_ref = (
    db
    .collection(colec_raiz)
    .document(doc_proyecto)
    .collection(colec_valoriz)
    .document(id_valoriz)
)

In [8]:
data_mantenimiento_res=doc_ref.get()

print(data_mantenimiento_res)

if not data_mantenimiento_res.exists:
    print(f"El documento {doc_ref.path} no existe.")

data_mantenimiento= data_mantenimiento_res.to_dict()
print(data_mantenimiento)

{'cronograma_desembolsos': {'11': {'mantenimiento_con_igv': 7438.1, 'mantenimiento_con_go': 8181.91, 'igv': 1338.86, 'gastos_operativos': 743.81, 'mantenimiento_sin_igv': 6099.24}, '5': {'mantenimiento_con_igv': 7686.04, 'mantenimiento_con_go': 8454.64, 'igv': 1383.49, 'gastos_operativos': 768.6, 'mantenimiento_sin_igv': 6302.55}, '6': {'mantenimiento_con_igv': 7438.1, 'mantenimiento_con_go': 8181.91, 'igv': 1338.86, 'gastos_operativos': 743.81, 'mantenimiento_sin_igv': 6099.24}, '12': {'mantenimiento_con_igv': 2727.29, 'mantenimiento_con_go': 3000.02, 'igv': 490.91, 'gastos_operativos': 272.73, 'mantenimiento_sin_igv': 2236.38}, '9': {'mantenimiento_con_igv': 7438.1, 'mantenimiento_con_go': 8181.91, 'igv': 1338.86, 'gastos_operativos': 743.81, 'mantenimiento_sin_igv': 6099.24}, '10': {'mantenimiento_con_igv': 7686.04, 'mantenimiento_con_go': 8454.64, 'igv': 1383.49, 'gastos_operativos': 768.6, 'mantenimiento_sin_igv': 6302.55}, '7': {'mantenimiento_con_igv': 7686.04, 'mantenimiento_co

#### Contrato

In [9]:
# 4. Define la ruta a tu documento anidado
colec_raiz = "rutinarios"
doc_proyecto = proyecto     # puede ser tu variable proyecto
colec_valoriz = "presupuestos"
id_valoriz = "desembolsos"  

doc_ref = (
    db
    .collection(colec_raiz)
    .document(doc_proyecto)
)

In [10]:
mantenimiento=doc_ref.get().to_dict()
contrato=mantenimiento['contrato']
print(contrato)

{'jefe_mantenimiento': {'nombre': 'Genaro', 'apellido': 'Tinta Cáceres', 'titulo': 'Ingeniero', 'dni': 0}, 'fecha_inicio': {'dia': 16, 'mes': 4, 'anio': 2025}, 'numero_trabajadores': 3, 'numero_cuadrillas': 1, 'denominacion_tramo_convenio': 'EMP. AR-691 CALLA CALLA - YURACCANCHA - CUCHUHURI - PALLALLMAYO (KM 11+920)', 'contratista': {'ruc': 20606988398, 'razon_social': 'Grupo ARICOL E.I.R.L'}, 'tiempo_ejecucion_dias': 240, 'id_contrato': '006-2025', 'monto_contrato': 59504.8, 'tipo_servicio': 'mantenimiento rutinario'}


In [11]:
monto_contrato=contrato['monto_contrato']
print(monto_contrato)

59504.8


#### Valorización programada mensual

In [12]:
# 4. Define la ruta a tu documento anidado
doc_proyecto = proyecto     # puede ser tu variable proyecto
colec_valoriz = "presupuestos"
id_valoriz = "desembolsos"  

doc_ref = (
    db
    .collection("rutinarios")
    .document(doc_proyecto)
    .collection(colec_valoriz)
    .document(id_valoriz)
)

desembolsos=doc_ref.get().to_dict()
cronograma_desembolsos=desembolsos['cronograma_desembolsos']
print(cronograma_desembolsos)



{'11': {'mantenimiento_con_igv': 7438.1, 'mantenimiento_con_go': 8181.91, 'igv': 1338.86, 'gastos_operativos': 743.81, 'mantenimiento_sin_igv': 6099.24}, '5': {'mantenimiento_con_igv': 7686.04, 'mantenimiento_con_go': 8454.64, 'igv': 1383.49, 'gastos_operativos': 768.6, 'mantenimiento_sin_igv': 6302.55}, '6': {'mantenimiento_con_igv': 7438.1, 'mantenimiento_con_go': 8181.91, 'igv': 1338.86, 'gastos_operativos': 743.81, 'mantenimiento_sin_igv': 6099.24}, '12': {'mantenimiento_con_igv': 2727.29, 'mantenimiento_con_go': 3000.02, 'igv': 490.91, 'gastos_operativos': 272.73, 'mantenimiento_sin_igv': 2236.38}, '9': {'mantenimiento_con_igv': 7438.1, 'mantenimiento_con_go': 8181.91, 'igv': 1338.86, 'gastos_operativos': 743.81, 'mantenimiento_sin_igv': 6099.24}, '10': {'mantenimiento_con_igv': 7686.04, 'mantenimiento_con_go': 8454.64, 'igv': 1383.49, 'gastos_operativos': 768.6, 'mantenimiento_sin_igv': 6302.55}, '7': {'mantenimiento_con_igv': 7686.04, 'mantenimiento_con_go': 8454.64, 'igv': 1383

In [29]:
desembolso_current_month=cronograma_desembolsos[str(mes)]
print(desembolso_current_month['mantenimiento_con_igv'])

7686.04


#### Cargas de trabajo mensual presentadas por el contratista

In [14]:
cargas_trabajo_contratista={
    "MR101":0.99,
    "MR301":3903.83,
    "MR103":5.01,
    "MR102":328.0,
    "MR203":44.45,
    "MR206":33.91,
    "MR104":10.17,
    "MR701":3.43
}

#### Cargas de trabajo y precios unitarios del expediente técnico

In [15]:
datos_cargas_trabajo_expediente_tecnico = {
    'MR101': {
        'precio_unitario': 265.30,
        'carga_trabajo': 7.95
    },
    'MR102': {
        'precio_unitario': 10.27,
        'carga_trabajo': 2558.43
    },
    'MR103': {
        'precio_unitario': 21.0,
        'carga_trabajo':  20.03
    },
    'MR104': {
        'precio_unitario': 17.50,
        'carga_trabajo': 71.20
    },
    'MR203':{
        'precio_unitario': 5.25,
        'carga_trabajo': 333.36
    },
    'MR206':{
        'precio_unitario': 3.6,
        'carga_trabajo': 237.37
    },
    'MR301': {
        'precio_unitario': 0.14,
        'carga_trabajo': 31230.64
    },
    'MR401':{
        'precio_unitario': 11.42,
        'carga_trabajo': 15.97
    },
    'MR601': {
        'precio_unitario': 2.10,
        'carga_trabajo': 39.73
    },
    'MR701': {
        'precio_unitario': 43.75,
        'carga_trabajo': 17.16
    }
}

## Cálculos

In [16]:
# Crear DataFrame usando pd.DataFrame.from_dict() con orient='index'
df_cargas_trabajo_expediente = pd.DataFrame.from_dict(datos_cargas_trabajo_expediente_tecnico, orient='index')

# Resetear el índice para convertir las claves MR en una columna
df_cargas_trabajo_expediente = df_cargas_trabajo_expediente.reset_index().rename(columns={'index': 'codigo_MR'})

In [17]:
# Agregar columna parcial
df_cargas_trabajo_expediente['parcial'] = df_cargas_trabajo_expediente['precio_unitario'] * df_cargas_trabajo_expediente['carga_trabajo']

In [18]:
df_cargas_trabajo_expediente.head()

,codigo_MR,precio_unitario,carga_trabajo,parcial
0,MR101,265.30,7.95,2109.1350
1,MR102,10.27,2558.43,26275.0761
2,MR103,21.00,20.03,420.6300
3,MR104,17.50,71.20,1246.0000
4,MR203,5.25,333.36,1750.1400


In [19]:
# Calcular el total
costo_directo = df_cargas_trabajo_expediente['parcial'].sum()
print("costo_directo",costo_directo)


costo_directo 38044.363099999995


In [20]:
costo_total_expediente=calculadora_costo_total(costo_directo)
print(costo_total_expediente['total'])


51626.2


In [21]:
# Agregar columna parcial
df_cargas_trabajo_expediente['precio_unitario_actualizado'] = df_cargas_trabajo_expediente['precio_unitario'] * (monto_contrato/costo_total_expediente['total'])

df_cargas_trabajo_expediente['parcial_actualizado'] = df_cargas_trabajo_expediente['precio_unitario_actualizado'] * df_cargas_trabajo_expediente['carga_trabajo']

# Calcular el total
costo_directo_actualizado = df_cargas_trabajo_expediente['parcial_actualizado'].sum()

print(f"El costo directo es: {costo_directo_actualizado}")

El costo directo es: 43850.258539130904


In [22]:
df_cargas_trabajo_expediente.head(10)

,codigo_MR,precio_unitario,carga_trabajo,parcial,precio_unitario_actualizado,parcial_actualizado
0,MR101,265.30,7.95,2109.1350,305.787051,2431.007054
1,MR102,10.27,2558.43,26275.0761,11.837290,30284.877607
2,MR103,21.00,20.03,420.6300,24.204780,484.821738
3,MR104,17.50,71.20,1246.0000,20.170650,1436.150265
4,MR203,5.25,333.36,1750.1400,6.051195,2017.226344
5,MR206,3.60,237.37,854.5320,4.149391,984.940897
6,MR301,0.14,31230.64,4372.2896,0.161365,5039.538416
7,MR401,11.42,15.97,182.3774,13.162790,210.209752
8,MR601,2.10,39.73,83.4330,2.420478,96.165590
9,MR701,43.75,17.16,750.7500,50.426624,865.320876


In [23]:
pago_costo_total_contratista=calculadora_costo_total(costo_directo_actualizado)
print(pago_costo_total_contratista['total'])

59504.8


In [24]:
dict_precios_unitarios_actualizados = dict(zip(
    df_cargas_trabajo_expediente['codigo_MR'],
    df_cargas_trabajo_expediente['precio_unitario_actualizado']
))

print(dict_precios_unitarios_actualizados)

{'MR101': 305.78705076104774, 'MR102': 11.837289903188692, 'MR103': 24.204779743618552, 'MR104': 20.170649786348793, 'MR203': 6.051194935904638, 'MR206': 4.1493908131917525, 'MR301': 0.16136519829079038, 'MR401': 13.162789746291613, 'MR601': 2.4204779743618556, 'MR701': 50.42662446587199}


#### Cálculo de pago de acuerdo a cargas de trabajo

In [32]:
pago_costo_directo_parciales_contratista=calcular_costos_unitarios(dict_precios_unitarios_actualizados,cargas_trabajo_contratista)
pprint.pprint(pago_costo_directo_parciales_contratista)

{'MR101': 302.7291802534373,
 'MR102': 3882.631088245891,
 'MR103': 121.26594651552894,
 'MR104': 205.13550832716723,
 'MR203': 268.97561490096115,
 'MR206': 140.7058424753323,
 'MR301': 629.9423020435362,
 'MR701': 172.96332191794093}


##### Visualizacion en dataframe

In [36]:
df_pago_costo_directo_parciales_contratista=pd.DataFrame.from_dict(
    pago_costo_directo_parciales_contratista, orient="index"
)

df_pago_costo_directo_parciales_contratista=df_pago_costo_directo_parciales_contratista.reset_index().rename(columns={'index':'codigo_MR'})

df_pago_costo_directo_parciales_contratista = (
    df_pago_costo_directo_parciales_contratista.rename(columns={0: "monto_pago"})
)

df_pago_costo_directo_parciales_contratista

,codigo_MR,monto_pago
0,MR101,302.729180
1,MR301,629.942302
2,MR103,121.265947
3,MR102,3882.631088
4,MR203,268.975615
5,MR206,140.705842
6,MR104,205.135508
7,MR701,172.963322


In [38]:
# Ordenar el DataFrame por la columna 'monto_pago'
# Por defecto, el orden es ascendente (de menor a mayor)
df_ordenado = df_pago_costo_directo_parciales_contratista.sort_values(
    by="monto_pago",ascending=False
)

print("\nDataFrame Ordenado por 'monto_pago' (ascendente):")
print(df_ordenado)


DataFrame Ordenado por 'monto_pago' (ascendente):
  codigo_MR   monto_pago
3     MR102  3882.631088
1     MR301   629.942302
0     MR101   302.729180
4     MR203   268.975615
6     MR104   205.135508
7     MR701   172.963322
5     MR206   140.705842
2     MR103   121.265947


In [33]:
# Lambda que suma todos los valores de un diccionario
sumar_valores = lambda d: sum(d.values())

In [34]:
pago_costo_directo_contratista = sumar_valores(pago_costo_directo_parciales_contratista)
print(pago_costo_directo_contratista)

5724.348804679795


In [28]:
pago_costo_total_contratista=calculadora_costo_total(pago_costo_directo_contratista)
print(pago_costo_total_contratista['total'])

7767.94
